# Linear Regression Model

# 1) importing key modules

In [1]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np 
import pandas as pd
import requests
import pickle

In [3]:
#For Visuals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [4]:
# for modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# Regression
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 

# Classification if needed
from sklearn.metrics import accuracy_score

#Model helper
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score

## 2)- Loading data-files

In [5]:
df = pd.read_csv('testset1.csv')

In [6]:
df.head(2)

,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,...,m16,m17,m18,m19,m20,m21,m22,m23,m24,m25
0,0.572565,-0.014704,-0.352985,-0.012697,-0.138549,0.620313,-0.602656,-0.071517,-0.131941,-0.067118,...,-0.199931,-0.360027,0.154202,0.369976,0.320812,1.252255,1.154858,0.609724,-0.126952,0.075709
1,-1.271232,0.754440,-0.206883,-0.572400,-0.303629,1.783537,0.338053,0.684332,0.780279,0.981463,...,1.112403,-0.317894,0.062038,-0.539969,-0.475836,0.734146,-0.377487,-0.449741,0.386476,0.419884


In [7]:
df.shape

(80000, 25)

# 3)- Model Preparation

### 3.1)-Backward elimination

In [8]:
# Automating backward elimination technique

def DoBackwardElimination(the_regressor, X, y, minP2eliminate):
    
    assert np.shape(X)[0] == np.shape(y)[0], 'Length of X and y do not match'
    assert minP2eliminate > 0, 'Minimum P value to eliminate cannot be zero or negative'
    
    original_list = list(range(0, np.shape(the_regressor.pvalues)[0]))
    
    max_p = 100        # Initializing with random value of maximum P value
    i = 0
    r2adjusted = []   # Will store R Square adjusted value for each loop
    r2 = []           # Will store R Square value  for each loop
    
    previous_R2adjusted = the_regressor.rsquared_adj
    
    while max_p >= minP2eliminate:
        
        p_values = list(the_regressor.pvalues)
        r2adjusted.append(the_regressor.rsquared_adj)
        r2.append(the_regressor.rsquared)
        
        max_p = max(p_values)
        max_p_idx = p_values.index(max_p)
        
        if max_p_idx == 0:
            
            temp_p = set(p_values)
            
            # removing the largest element from temp list
            temp_p.remove(max(temp_p))
            
            max_p = max(temp_p)
            max_p_idx = p_values.index(max_p)
            
#             print('Index value 0 found!! Next index value is {}'.format(max_p_idx))
            
            if max_p < minP2eliminate:
                
                print('Max P value found less than 0.1 with 0 index ...Loop Ends!!')
                
                break
                
        if max_p < minP2eliminate:
            
            print('Max P value found less than 0.1 without 0 index...Loop Ends!!')
            
            break
        
        val_at_idx = original_list[max_p_idx]
        
        idx_in_org_lst = original_list.index(val_at_idx)
        
        original_list.remove(val_at_idx)
        
        print('Popped column index out of original array is {} with P-Value {}'.format(val_at_idx, np.round(np.array(p_values)[max_p_idx], decimals= 4)))
        
        print('==================================================================================================')
        
        X_new = X[:, original_list]
        
        the_regressor = smf.OLS(endog = y, exog = X_new).fit()
        
        if previous_R2adjusted < the_regressor.rsquared_adj:
            classifier_with_maxR2adjusted = the_regressor
            final_list_of_index = copy.deepcopy(original_list)
            previous_R2adjusted = the_regressor.rsquared_adj
            
    return classifier_with_maxR2adjusted, r2, r2adjusted, final_list_of_index


### 3.2)-Plot residuals

In [9]:
def resiplot(y_original, y_predicted, delete_outlier = False, max_outlier_val = None):
    
    residual = y_original - y_predicted
    residnew = list(residual.ravel())
    
    if delete_outlier == True:
        assert max_outlier_val != None, 'Please insert \'max_outlier_val\''
        count = 0
        while max(residnew) > abs(max_outlier_val):
            count = count + 1
            residnew.remove(max(residnew))
            
        while min(residnew)< -abs(max_outlier_val):
            count = count + 1
            residnew.remove(min(residnew))
        print('Residuals with unreal values are {} i.e. only {}% of total test data.'.format(count, np.round((count/len(residnew)*100), 2)))

    plt.scatter(x = range(0, len(residnew)), y = residnew, s = 2, c = 'R')
    plt.plot([0,len(residnew)], [0,0], '-k')
    if delete_outlier == True:
        plt.ylim(-abs(max_outlier_val), abs(max_outlier_val))
    elif abs(max(residnew)) > abs(min(residnew)):
        plt.ylim(-max(residnew), max(residnew))
    else:
        plt.ylim(min(residnew), abs(min(residnew)))
        
    plt.title('Mean residual is {}'.format(np.round(np.mean(residnew),2)))

### 3.3) Splitting data into X and y

In [10]:
# Selecting all columns except last one that is 'm25'.

X = df.iloc[:,:-1].values          
y = df['m25'].values

In [11]:
X

array([[ 0.57256488, -0.01470396, -0.35298469, ...,  1.1548579 ,
         0.60972403, -0.12695157],
       [-1.27123156,  0.75443995, -0.20688296, ..., -0.37748682,
        -0.44974088,  0.38647552],
       [-0.77836448, -0.45371563, -2.10020124, ...,  0.18666614,
         0.21163226, -0.56662082],
       ...,
       [ 0.73740991, -0.5590069 ,  0.14355845, ..., -0.98605973,
         0.20485423, -0.16246918],
       [ 0.1151475 , -1.04815064,  0.65888561, ...,  1.15475919,
         1.25093225,  1.49547161],
       [ 0.3145309 ,  0.77643942,  0.96305974, ..., -0.15680151,
         1.68539657,  0.16798312]])

In [12]:
X.shape

(80000, 24)

In [13]:
y

array([ 0.07570943,  0.41988445, -1.00067051, ..., -0.55195226,
        0.32491959, -0.52513385])

In [14]:
print(type(X))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2019)

# 4) Applying Model

In [16]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [17]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

### 4.1)-Using econometrics concepts

Apply statsmodels for regression model

In [18]:
import statsmodels.formula.api as sm
from statsmodels.regression.linear_model import OLS
X = np.append(arr=np.ones((80000, 1)).astype(int), values=X, axis=1)
X_opt = X[:, [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     55.03
Date:                Wed, 07 Aug 2019   Prob (F-statistic):          7.98e-262
Time:                        17:26:15   Log-Likelihood:            -1.0525e+05
No. Observations:               80000   AIC:                         2.106e+05
Df Residuals:                   79975   BIC:                         2.108e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0048      0.003     -1.500      0.134      -0.011       0.001
x1            -0.0003      0.004     -0.080      0.937      -0.007       0.007
x2            -0.0011      0.004     -0.296      0.767      -0.008       0.006
x3            -0.0028      0.004     -0.776      0.438      -0.010       0.004
x4            -0.0038      0.004     -1.075      0.283      -0.011       0.003
x5          8.454e-05      0.004      0.024      0.981      -0.007       0.007
x6             0.0037      0.004      1.044      0.296      -0.003       0.011
x7            -0.0002      0.004     -0.051      0.959      -0.007       0.007
x8            -0.0016      0.004     -0.447      0.655      -0.009       0.005
x9            -0.0019      0.004     -0.541      0.589      -0.009       0.005
x10            0.0028      0.004      0.779      0.436      -0.004       0.010
x11           -0.0010      0.004     -0.278      0.781      -0.008       0.006
x12            0.0010      0.004      0.281      0.779      -0.006       0.008
x13            0.0272      0.004      7.674      0.000       0.020       0.034
x14            0.0235      0.004      6.638      0.000       0.017       0.030
x15            0.0260      0.004      7.290      0.000       0.019       0.033
x16            0.0386      0.004     10.841      0.000       0.032       0.046
x17            0.0328      0.004      9.243      0.000       0.026       0.040
x18            0.0341      0.004      9.580      0.000       0.027       0.041
x19            0.0282      0.004      7.914      0.000       0.021       0.035
x20            0.0298      0.004      8.399      0.000       0.023       0.037
x21            0.0284      0.004      8.030      0.000       0.021       0.035
x22            0.0277      0.004      7.835      0.000       0.021       0.035
x23            0.0325      0.004      9.168      0.000       0.026       0.039
x24            0.0340      0.004      9.571      0.000       0.027       0.041
==============================================================================
Omnibus:                        3.757   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.153   Jarque-Bera (JB):                3.787
Skew:                          -0.009   Prob(JB):                        0.151
Kurtosis:                       3.028   Cond. No.                         1.44
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**We will take into account only those variables whose p-value is less than 0.05 i.e 5%. All those variables from 13-24 are significant. Rest do not satisfy our p-values so, we will consider them insignificant**

### 4.2)-backward elimation

In [19]:
X_opt = X[:, [13, 14, 15, 16, 17,18,19,20,21,22,23,24]]
#regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
#regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     109.7
Date:                Wed, 07 Aug 2019   Prob (F-statistic):          2.67e-272
Time:                        17:26:15   Log-Likelihood:            -1.0526e+05
No. Observations:               80000   AIC:                         2.105e+05
Df Residuals:                   79988   BIC:                         2.106e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0270      0.004      7.650      0.000       0.020       0.034
x2             0.0234      0.004      6.623      0.000       0.016       0.030
x3             0.0259      0.004      7.283      0.000       0.019       0.033
x4             0.0386      0.004     10.857      0.000       0.032       0.046
x5             0.0328      0.004      9.289      0.000       0.026       0.040
x6             0.0341      0.004      9.606      0.000       0.027       0.041
x7             0.0281      0.004      7.919      0.000       0.021       0.035
x8             0.0297      0.004      8.402      0.000       0.023       0.037
x9             0.0284      0.004      8.045      0.000       0.021       0.035
x10            0.0278      0.004      7.879      0.000       0.021       0.035
x11            0.0325      0.004      9.178      0.000       0.026       0.039
x12            0.0340      0.004      9.587      0.000       0.027       0.041
==============================================================================
Omnibus:                        3.720   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.156   Jarque-Bera (JB):                3.750
Skew:                          -0.009   Prob(JB):                        0.153
Kurtosis:                       3.028   Cond. No.                         1.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We got a very low R2 value.

# 5)- Re-applying model after backward elimation

In [20]:
clf_lr = LinearRegression()
clf_lr.fit(X_opt , y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [21]:
y_pred = clf_lr.predict(X_test)

In [22]:
y_pred[:5]

array([ 0.13891801,  0.07357355,  0.05625965, -0.13652366, -0.11772136])

In [23]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

In [24]:
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse)
print('R2     : %0.2f ' % r2)

MSE    : 0.82 
MAE    : 0.72 
RMSE   : 0.90 
R2     : 0.02 



We got some improvment there in R2 as we removed non-significant features. Earlier, it was 0.016 and now we have 0.02